In [1]:
from langchain.llms import OpenAI

In [2]:
llm = OpenAI(model="gpt-3.5-turbo-instruct", temperature=0.9)

In [3]:
text = "Suggest a personalized workout routine for someone looking to improve cardiovascular endurance and prefers outdoor activities."
print(llm(text))



Day 1:
Warm-up: 5 minute brisk walk or jog
Activity: 30 minutes of outdoor cycling
Cool down: 5 minute slow bike ride

Day 2:
Warm-up: 5 minute light jog
Activity: 20 minute outdoor HIIT (high intensity interval training) workout (e.g. sprints, burpees, jumping jacks)
Cool down: 5 minute walk

Day 3:
Warm-up: 5 minute brisk walk
Activity: 30 minute outdoor hike or trail run
Cool down: 5 minute walk or light jog

Day 4:
Rest day or active recovery activity such as yoga or stretching.

Day 5:
Warm-up: 5 minute light jog
Activity: 40 minute outdoor swimming or water aerobics
Cool down: 5 minute walk or jog in the water

Day 6:
Warm-up: 5 minute brisk walk
Activity: 30 minute outdoor circuit training (e.g. bodyweight exercises like push-ups, squats, lunges, and planks)
Cool down: 5 minute walk

Day 7:
Rest day or active recovery activity such as cycling or light yoga.

Note: Repeat this workout routine for 2-3 weeks before switching


In [4]:
from langchain.prompts import PromptTemplate
from langchain.llms import OpenAI
from langchain.chains import LLMChain

llm = OpenAI(model="gpt-3.5-turbo-instruct", temperature=0.9)
prompt = PromptTemplate(
    input_variables=["product"],
    template="What is a good name for a company that makes {product}?",
)

chain = LLMChain(llm=llm, prompt=prompt)

# Run the chain only specifying the input variable.
print(chain.run("eco-friendly water bottles"))



1. "EcoH2O" 
2. "SustainaBottle" 
3. "GreenWave Bottles" 
4. "NatureNest Water" 
5. "EcoQuench" 
6. "PlanetPure Bottles" 
7. "HydroGreen Co." 
8. "EcoAqua Solutions" 
9. "PureEarth Bottles" 
10. "Eco-Sip"


In [5]:
from langchain.chains import ConversationChain
from langchain.memory import ConversationBufferMemory

llm = OpenAI(model="gpt-3.5-turbo-instruct", temperature=0)
conversation = ConversationChain(
    llm=llm,
    verbose=True,
    memory=ConversationBufferMemory()
)

# Start the conversation
conversation.predict(input="Tell me about yourself.")

# Continue the conversation
conversation.predict(input="What can you do?")
conversation.predict(input="How can you help me with data analysis?")

# Display the conversation
print(conversation)



> Entering new  chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:

Human: Tell me about yourself.
AI:

> Finished chain.


> Entering new  chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
Human: Tell me about yourself.
AI:  Well, I am an artificial intelligence program designed and created by a team of programmers. I am constantly learning and improving my abilities through algorithms and data analysis. My main purpose is to assist and provide information to users like yourself. I am currently housed in a server loc

In [8]:
import os
os.environ["ACTIVELOOP_TOKEN"] = "eyJhbGciOiJub25lIiwidHlwIjoiSldUIn0.eyJpZCI6InJhdXRtb2hpdDA4IiwiYXBpX2tleSI6IkNOMkFrdXdNeDhweXVIQVZFcnJNNi0xd1R0T1IxdXcyaTdmYkRKUFVzQW43RyJ9."

In [9]:
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.vectorstores import DeepLake
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.llms import OpenAI
from langchain.chains import RetrievalQA

# Before executing the following code, make sure to have your
# Activeloop key saved in the “ACTIVELOOP_TOKEN” environment variable.

# instantiate the LLM and embeddings models
llm = OpenAI(model="gpt-3.5-turbo-instruct", temperature=0)
embeddings = OpenAIEmbeddings(model="text-embedding-ada-002")

# create our documents
texts = [
    "Napoleon Bonaparte was born in 15 August 1769",
    "Louis XIV was born in 5 September 1638"
]
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
docs = text_splitter.create_documents(texts)

# create Deep Lake dataset
# TODO: use your organization id here. (by default, org id is your username)
my_activeloop_org_id = "rautmohit08" 
my_activeloop_dataset_name = "langchain_course_from_zero_to_hero"
dataset_path = f"hub://{my_activeloop_org_id}/{my_activeloop_dataset_name}"
db = DeepLake(dataset_path=dataset_path, embedding_function=embeddings)

# add documents to our Deep Lake dataset
db.add_documents(docs)

Your Deep Lake dataset has been successfully created!


Creating 2 embeddings in 1 batches of size 2:: 100%|██████████| 1/1 [00:28<00:00, 28.72s/it]

Dataset(path='hub://rautmohit08/langchain_course_from_zero_to_hero', tensors=['text', 'metadata', 'embedding', 'id'])

  tensor      htype      shape     dtype  compression
  -------    -------    -------   -------  ------- 
   text       text      (2, 1)      str     None   
 metadata     json      (2, 1)      str     None   
 embedding  embedding  (2, 1536)  float32   None   
    id        text      (2, 1)      str     None   


['38b33d5e-d92d-11ee-a0b7-0028f87be246',
 '38b33d5f-d92d-11ee-bafb-0028f87be246']

In [10]:
retrieval_qa = RetrievalQA.from_chain_type(
	llm=llm,
	chain_type="stuff",
	retriever=db.as_retriever()
)

In [11]:
from langchain.agents import initialize_agent, Tool
from langchain.agents import AgentType

tools = [
    Tool(
        name="Retrieval QA System",
        func=retrieval_qa.run,
        description="Useful for answering questions."
    ),
]

agent = initialize_agent(
	tools,
	llm,
	agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
	verbose=True
)

In [12]:
response = agent.run("When was Napoleone born?")
print(response)



> Entering new  chain...
 I should use the Retrieval QA System to answer this question
Action: Retrieval QA System
Action Input: When was Napoleone born?
Observation: 
Napoleon Bonaparte was born in 15 August 1769.
Thought: I now know the final answer
Final Answer: Napoleon Bonaparte was born in 15 August 1769.

> Finished chain.
Napoleon Bonaparte was born in 15 August 1769.


In [13]:
# load the existing Deep Lake dataset and specify the embedding function
db = DeepLake(dataset_path=dataset_path, embedding_function=embeddings)

# create new documents
texts = [
    "Lady Gaga was born in 28 March 1986",
    "Michael Jeffrey Jordan was born in 17 February 1963"
]
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
docs = text_splitter.create_documents(texts)

# add documents to our Deep Lake dataset
db.add_documents(docs)

Deep Lake Dataset in hub://rautmohit08/langchain_course_from_zero_to_hero already exists, loading from the storage


Creating 2 embeddings in 1 batches of size 2:: 100%|██████████| 1/1 [01:11<00:00, 71.35s/it]

Dataset(path='hub://rautmohit08/langchain_course_from_zero_to_hero', tensors=['embedding', 'id', 'metadata', 'text'])

  tensor      htype      shape     dtype  compression
  -------    -------    -------   -------  ------- 
 embedding  embedding  (4, 1536)  float32   None   
    id        text      (4, 1)      str     None   
 metadata     json      (4, 1)      str     None   
   text       text      (4, 1)      str     None   


['9f88f82d-d92d-11ee-bf65-0028f87be246',
 '9f88f82e-d92d-11ee-88eb-0028f87be246']

In [14]:
# instantiate the wrapper class for GPT3
llm = OpenAI(model="gpt-3.5-turbo-instruct", temperature=0)

# create a retriever from the db
retrieval_qa = RetrievalQA.from_chain_type(
	llm=llm, chain_type="stuff", retriever=db.as_retriever()
)

# instantiate a tool that uses the retriever
tools = [
    Tool(
        name="Retrieval QA System",
        func=retrieval_qa.run,
        description="Useful for answering questions."
    ),
]

# create an agent that uses the tool
agent = initialize_agent(
	tools,
	llm,
	agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
	verbose=True
)   

In [15]:
response = agent.run("When was Michael Jordan born?")
print(response)



> Entering new  chain...
 I should use the Retrieval QA System to find the answer.
Action: Retrieval QA System
Action Input: "When was Michael Jordan born?"
Observation:  Michael Jordan was born on 17 February 1963.
Thought: I now know the final answer.
Final Answer: Michael Jordan was born on 17 February 1963.

> Finished chain.
Michael Jordan was born on 17 February 1963.
